In [1]:
import numpy as np
import quimb as qu
import quimb.tensor as qtn

import cotengra as ctg

import torch
import floquet

from floquet.circuits import (trotter_2D_square, circ_gates, Z1_ti, Z2_ti, mpo_z2_, simulate, transpile, weights)
from tqdm import tqdm 
import algo_cooling as algo
import algo_dmrg
import register_ as reg
from quimb.tensor.belief_propagation.l2bp import L2BP
import quf
import math

ModuleNotFoundError: No module named 'floquet'

In [ ]:
import torch
to_backend = algo.backend_torch(device = "cpu", dtype = torch.complex128)
to_backend_ = algo.backend_numpy( dtype = "complex128")

# opt = algo.opt_(progbar=True, target_size=2**32, subtree_size=12)
opt = algo.opt_(progbar=True, max_time="rate:1e9", target_size=2**32, 
                 subtree_size=12, max_repeats=2**8, optlib="cmaes")


In [ ]:
Lx, Ly = 4, 4  # Lx < Ly
L = Lx * Ly
steps = 22
gate_info, depth_map, circ_info = quf.circ_info(Lx=Lx, Ly=Ly, steps=steps)
mpoz2 = mpo_z2_(circ_info, "Z", where_=L//2, form="left")
mpoz2.apply_to_arrays(to_backend)
mpoz2.show()

In [ ]:
res_su = {}
res_su = qu.load_from_disk("store/res_su")
res_su.setdefault(f"L_{L}", [])
res_su.keys()

In [ ]:
z2_exact = qu.load_from_disk(f"z2_exact/z2_ext_Lx{Lx}Ly{Ly}")
# z2_exact

In [ ]:
L = Lx * Ly
# vacuum state
p = qtn.TN_from_sites_product_state({site: [1.0, 0.0] for site in range(L)})
site_tags = [f"I{n}" for n in range(L)]
# create size 1 bonds
for i in range(L):
    p[i].new_bond(p[(i+1)%L])

# for i in range(L-1):
#     p[i].new_bond(p[i+1])


p.apply_to_arrays(to_backend)


In [ ]:
# create simple update gauges
gauges = {}
p.gauge_all_simple_(gauges=gauges, progbar=True)

In [ ]:
def gen_long_range_swap_path(L, x, y):
    """Return forward/backward paths between x and y on a ring of length L,
    but as link pairs: (site_i, site_i+1)."""

    # distances around the ring
    forward_dist = (y - x) % L
    backward_dist = (x - y) % L


    # shortest
    if forward_dist <= backward_dist:
        # forward: x → x+1 → ... → y
        forward_nodes = [(x + i) % L for i in range(0, forward_dist )]
        forward_pairs = [ (forward_nodes[i], forward_nodes[i+1])
                          for i in range(len(forward_nodes)-1) ]
        return forward_pairs
        
    elif backward_dist < forward_dist:
        backward_nodes = [(x - i) % L for i in range(0, backward_dist )]
        backward_pairs = [ (backward_nodes[i], backward_nodes[i+1])
                           for i in range(len(backward_nodes)-1) ]
        return backward_pairs


def gen_long_range_swap_path_(L, x, y):

    # distances around the ring
    forward_dist = (y - x) % L

    # forward: x → x+1 → ... → y
    forward_nodes = [(x + i) % L for i in range(0, forward_dist )]
    forward_pairs = [ (forward_nodes[i], forward_nodes[i+1])
                      for i in range(len(forward_nodes)-1) ]
    return forward_pairs
        
def energy_global_(MPO_origin, mps_a, opt="auto-hq"):

    norm_ = (mps_a.H | mps_a).contract(all, optimize=opt)
    
    p_h=mps_a.H 
    p_h.reindex_(  { f"k{i}":f"b{i}" for i in range(L)} )
  
    E_dmrg = (p_h | MPO_origin | mps_a).contract(all,optimize=opt)
    
    return E_dmrg / norm_, norm_


In [ ]:
# path_ = gen_long_range_swap_path_(L, 6, 12)
# path_

In [ ]:
%%time
chi = 32
F_l, Fidel_l, Norm_l, norm, tqg_, f = ([], [1], [1], 1, 0, 1)
e_x_ = {}

swap = to_backend(qu.swap(dim=2, dtype="complex128"))
swap = swap.reshape(2,2,2,2)
equilib = True

e_x_ = {}
res = {f"F_L{L}_chi{chi}":[], f"Scale_L{L}_chi{chi}":[], f"Z2_L{L}_chi{chi}":[], f"Z2_L{L}":z2_exact}
res |= {f"F_L{L}_chi{chi}":[], f"Norm_L{L}_chi{chi}":[], f"Gamma_L{L}_chi{chi}":[]}


with tqdm(total=len(gate_info),  desc="dmrg:", leave=True, position=0, 
        colour='CYAN', disable = not True) as pbar:

    for (where, count), G in gate_info.items():

        if len(where) == 1:
            p.gate_simple_(to_backend(G), where, gauges=gauges)

        if len(where) == 2:

            if equilib:
                p.gauge_all_simple_(max_iterations=100, tol=1e-7, gauges=gauges, progbar=False)
            
            tqg_ += 1
            
            start, end = where
            if end < start:
                print("warning", start, end)
            
            if abs(start-end) == 1:
                p.gate_simple_(to_backend(G), where, gauges=gauges, max_bond=chi, cutoff=1e-12, cutoff_mode="rel", renorm=False)
            else:
                swaps = gen_long_range_swap_path(L, start, end)
                
                for pair in swaps:
                    p.gate_simple_(swap, pair,gauges=gauges, max_bond=chi, cutoff=1e-12,
                                   cutoff_mode="rel",renorm=False)

                if equilib:
                    p.gauge_all_simple_(max_iterations=100, tol=1e-7, gauges=gauges, progbar=False)

                
                _, start_ = swaps[-1]
                p.gate_simple_(to_backend(G), (start_, end), gauges=gauges, max_bond=chi, cutoff=1e-12,
                               cutoff_mode="rel",renorm=False)

                if equilib:
                    p.gauge_all_simple_(max_iterations=100, tol=1e-7, gauges=gauges, progbar=False)
                
                for pair in reversed(swaps):
                    p.gate_simple_(swap, pair,gauges=gauges, max_bond=chi, cutoff=1e-12,
                                   cutoff_mode="rel",renorm=False)
 

 
        
        pbar.set_postfix({"tqg_":tqg_, })
        pbar.refresh()
        pbar.update(1)
        
        # measure an observable
        if count+1 in list(depth_map.keys()):
            p_ = p.copy()
            gauges_ = { key: value*1.  for key, value in gauges.items()}
            p_.gauge_simple_insert(gauges_)
            depth = depth_map[count+1]
            e_x, norm_ = energy_global_(mpoz2, p_, opt=opt)
            e_x = complex(e_x).real
            e_x_[depth] = e_x
            res[f"F_L{L}_chi{chi}"].append(  complex(norm_**2).real )
            res[f"Scale_L{L}_chi{chi}"].append( z2_exact[depth]/ e_x )
            res[f"Norm_L{L}_chi{chi}"].append( complex(norm_).real )
            res[f"Z2_L{L}_chi{chi}"].append( e_x )
            res[f"Gamma_L{L}_chi{chi}"].append( np.log(z2_exact[depth]/ e_x) / (np.log(complex(norm_).real)+1.e-12) )


In [ ]:
# res

In [ ]:
p_ = p.copy()
gauges_ = { key: value*1.  for key, value in gauges.items()}
# the state normalization gives us an idea of fidelity
p_.normalize_simple(gauges_)

In [ ]:
p.draw()

In [ ]:
p.gauge_simple_insert(gauges)
(p.H & p).contract(all, optimize=opt)

In [ ]:
# boundary propagation norm estimate
bp = L2BP(p, optimize=opt, site_tags=site_tags)
bp.run(max_iterations=2_000, tol=1.e-7, progbar=False, diis=True)
est_norm = complex(bp.contract()).real
est_norm

In [ ]:
res_su |= res
print(res_su.keys())
res_su[f"L_{L}"].append(chi)
qu.save_to_disk(res_su, "store/res_su")